In [1]:
#Importar bibliotecas
import requests as rq
import json
import pandas as pd
import numpy as np
import psycopg2 as pg2

#Resumo do que é feito nesse código:
#Extração de dados do API
#Transformação de dados do tipo JSON para relacional (DataFrame)
#Criação de tabelas no PostgreSQL e inserção dos dados no banco
#0s dados podem ser vizualizados nos data frames do python e no postgres (pgAdmin4)
#É preciso antes instalar o postgres (pgAdmin4) e criar a base de dados "dados_filmes"

In [2]:
#Função para extrair dados do API
#Criei para esconder variaveis indesejadas
def formar_frames():
    #Criando frames para armazenar dados
    #Armazena filmes
    filmes=pd.DataFrame() 
    #Armazena os IDs dos filmes e seus respectivos IDs de generos
    generos_filmes=pd.DataFrame()
    genre=pd.DataFrame()
    #Armanezando informações sobre as companias
    companias=pd.DataFrame()
    company=pd.DataFrame()
    #Extraindo 1000 filmes do api
    for i in range(1,1547):   
        #Conectando com API em forma de dicionário
        dict=rq.get(f'https://api.themoviedb.org/3/movie/{i}?api_key=882abf7846ec728e7e45c208a6ece74d').json()
        #Normalizando dicionário em frame
        frame=pd.json_normalize(dict)
        #Adicionando dados no data frame filmes
        filmes=pd.concat([filmes,frame])
        #Tentar achar a categoria 'genres' no dicionário
        try:
            #Extraindo informação de generos do filme
            genre=pd.DataFrame(dict['genres'])
            #Renomeando colunas
            genre.rename(columns={'id':'genre_id','name':'genre_name'},inplace=True)
            #Adicionando coluna como id do filme
            genre['film_id']=[dict['id']]*len(genre)
            #Mudando ordem
            genre=genre[['genre_id','film_id','genre_name']]
        except Exception:
            genre=pd.DataFrame()
        #Adicionando dados em data frame de generos
        generos_filmes=pd.concat([generos_filmes,genre])
        #Tentar achar a categoria 'production_companies' no dicionário
        try:
            #Extraindo informação sobre a empresa
            company=pd.DataFrame(dict['production_companies'])
            #Retirando coluna indesejada
            company.drop(['logo_path'],inplace=True,axis=1)
            #Preenchendo valor vazio com nan
            company['origin_country'].replace('', np.nan, regex=True,inplace=True)
            #Renomeando colunas
            company.rename(columns={'id':'company_id','name':'company_name'},inplace=True)
            #Adicionando coluna com o id do respectivo filme
            company['film_id']=[dict['id']]*len(company)
            #Mudando ordem
            company=company[['company_id','film_id','company_name','origin_country']]
        except Exception:
            company=pd.DataFrame()
        companias=pd.concat([companias,company])
    #Retirando filmes sem ID (são filmes sem valores ou com valores corrompidos em suas colunas)
    filmes.dropna(subset=['id'],inplace=True)
    filmes=filmes[['id','title','original_language','popularity','runtime','vote_average','vote_count','overview','revenue','budget','release_date']]
    filmes.rename(columns={'id':'film_id'},inplace=True)
    return filmes, generos_filmes, companias 

In [3]:
#Função para criar tabela de filmes no PostgreSQL
def criar_tabela_postgre_film():
        #Conectando com o PostgreSQL
        conn = pg2.connect(database="dados_filmes", user="postgres", password="200613")
        cur=conn.cursor()
        #Query no PostgreSQL
        cur.execute('''
                CREATE TABLE IF NOT EXISTS filmes (
                film_id INTEGER PRIMARY KEY,
                title varchar,
                original_language varchar,
                popularity FLOAT,
                runtime FLOAT,
                vote_average FLOAT,
                vote_count FLOAT,
                overview varchar,
                revenue FLOAT,
                budget FLOAT,
                release_date TIMESTAMP)
                ''')
        #Fechando conexão
        cur.close()
        conn.commit()

In [4]:
#Função para criar tabela de generos de filmes no PostgreSQL
def criar_tabela_postgre_generos_filmes():
        #Conectando com o PostgreSQL
        conn = pg2.connect(database="dados_filmes", user="postgres", password="200613")
        cur=conn.cursor()
        #Query no PostgreSQL
        cur.execute('''
                CREATE TABLE IF NOT EXISTS generos_filmes (
                genre_id INTEGER NOT NULL,
                film_id INTEGER NOT NULL,
                genre_name varchar)
                ''')
        #Fechando conexão
        cur.close()
        conn.commit()

In [5]:
#Função para criar tabela de companias produtoras no PostgreSQL
def criar_tabela_postgre_companias():
        #Conectando com o PostgreSQL
        conn = pg2.connect(database="dados_filmes", user="postgres", password="200613")
        cur=conn.cursor()
        #Query no PostgreSQL
        cur.execute('''
                CREATE TABLE IF NOT EXISTS companias (
                company_id INTEGER NOT NULL,
                film_id INTEGER NOT NULL,
                company_name varchar,
                origin_country varchar)
                ''')
        #Fechando conexão
        cur.close()
        conn.commit()

In [6]:
#Função para inserir dados na tabela de filmes do PostgreSQL
def inserir_dados_filmes(filmes):
    #Conectando com o PostgreSQL
    conn = pg2.connect(database="dados_filmes", user="postgres", password="200613")
    cur=conn.cursor()
    #Query no PostgreSQL
    insert_query = f"""insert into filmes(
        film_id,
        title,
        original_language,
        popularity,
        runtime,
        vote_average,
        vote_count,
        overview,
        revenue,
        budget,
        release_date) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    cur.executemany(insert_query, filmes.values)
    #Fechando conexão
    cur.close()
    conn.commit()         

In [7]:
#Função para inserir dados na tabela de generos do PostgreSQL
def inserir_dados_generos(generos_filmes):
    #Conectando com o PostgreSQL
    conn = pg2.connect(database="dados_filmes", user="postgres", password="200613")
    cur=conn.cursor()
    #Query no PostgreSQL
    insert_query = f"""insert into generos_filmes(
        genre_id,
        film_id,
        genre_name) values (%s,%s,%s)"""
    cur.executemany(insert_query, generos_filmes.values)
    #Fechando conexão
    cur.close()
    conn.commit()   

In [8]:
#Função para inserir dados na tabela de companias do PostgreSQL
def inserir_dados_companias(companias):
    #Conectando com o PostgreSQL
    conn = pg2.connect(database="dados_filmes", user="postgres", password="200613")
    cur=conn.cursor()
    #Query no PostgreSQL
    insert_query = f"""insert into companias(
        company_id,
        film_id,
        company_name,
        origin_country) values (%s,%s,%s,%s)"""
    cur.executemany(insert_query, companias.values)
    #Fechando conexão
    cur.close()
    conn.commit()  

In [9]:
#Executando funções
#Criando tabelas na base de dados PostgreSQL
#OBS:Deixar base de dados já criada no software pgAdmin4
criar_tabela_postgre_generos_filmes()
criar_tabela_postgre_companias()
criar_tabela_postgre_film()
#Extraindo informações do API e transformando em tabelas
filmes, generos_filmes, companias = formar_frames()

In [10]:
#Preenchendo dados nas tabelas do PostgreSQL
inserir_dados_filmes(filmes)
inserir_dados_generos(generos_filmes)
inserir_dados_companias(companias)

In [ ]:
#Exportar como excel
writer = pd.ExcelWriter('Filmes.xlsx', engine='xlsxwriter')
#Escrevendo guias
filmes.to_excel(writer,sheet_name='Filmes',index=False)
generos_filmes.to_excel(writer,sheet_name='Generos_filmes',index=False)
companias.to_excel(writer,sheet_name='Companias',index=False)
#Salvando
writer.save()